In [0]:

# "aa.usage.basic-kpi.v6", s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=ios_diamond_1.0.0/ day week month
# "aa.usage.audience.v6", s3://aardvark-prod-pdx-mdm-to-int/to_tech/audience/version=ios_diamond_1.0.0/   month
# "aa.usage.seg-by-age-gender.v6", s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=ios_diamond_1.0.0/ month week
# "aa.usage.app-cross-app.v6",  s3://aardvark-prod-pdx-mdm-to-int/to_tech/crossapps/version=ios_diamond_1.0.0/ month
# "aa.usage.seg-by-product.v6", s3://aardvark-prod-pdx-mdm-to-int/to_tech/appused/version=ios_diamond_1.0.0/ MONTH WEEK
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_BASIC_METRICS/version=ios_diamond_1.0.0": "aa.usage.domain.basic.v6",
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_REFERRAL_METRICS/version=ios_diamond_1.0.0": "aa.usage.domain-referral.v6",
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_OUTBOUND_METRICS/version=ios_diamond_1.0.0": "aa.usage.domain-outbound.v6",
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_DOMAIN_METRICS/version=ios_diamond_1.0.0": "aa.usage.domain-cross-domain.v6",
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_CROSS_APP_METRICS/version=ios_diamond_1.0.0": "aa.usage.domain-cross-app.v6",
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_APP_CROSS_DOMAIN_METRICS/version=ios_diamond_1.0.0": "aa.usage.app-cross-domain.v6",
# "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_UNIFIED_PRODUCT_METRICS/version=ios_diamond_1.0.0": "aa.usage.unified-attribution.v6",
# spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=ios_diamond_1.0.0/range_type=WEEK/date=2018-06-09/").createOrReplaceTempView("test")
# spark.sql("select * from test").show(20)
# df = spark.sql("select gender,age,count(*),sum(AU) from test group by gender,age".format(date='2021-01-01', granularity='MONTH')).show(20)

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "daily": get_date_list(begin_date, end_date, "D"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT"),
        "monthly": get_date_list(begin_date, end_date, "M")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=fix_crow_1.0.0/range_type={range_type}/date={date}/platform=2/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_basic_kpi")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(AU) as sum from test_basic_kpi".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-08-13/basic_kpi/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))

In [0]:


# "aa.usage.basic-kpi.v6", s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=ios_diamond_1.0.0/ day week month
# "aa.usage.audience.v6", s3://aardvark-prod-pdx-mdm-to-int/to_tech/audience/version=ios_diamond_1.0.0/   month
# "aa.usage.seg-by-age-gender.v6", s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=ios_diamond_1.0.0/ month week
# "aa.usage.app-cross-app.v6",  s3://aardvark-prod-pdx-mdm-to-int/to_tech/crossapps/version=ios_diamond_1.0.0/ month
# "aa.usage.seg-by-product.v6", s3://aardvark-prod-pdx-mdm-to-int/to_tech/appused/version=ios_diamond_1.0.0/ MONTH WEEK

# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/basic_kpi/").show(3)
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/audience/").show(3)
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/demographics/").show(3)
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/crossapps/").show(3)
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/appused/").show(3)

date='2021-08-13'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/basic_kpi/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/basic_kpi.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/audience/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/audience.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/demographics/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/demographics.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/crossapps/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/crossapps.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/appused/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/appused.csv".format(date),header = 'true')

# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/domain_outbound/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/domain_outbound.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/domain_referral/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/domain_referral.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/test_app_cross_domain/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/test_app_cross_domain.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/test_domain_cross_app/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/test_domain_cross_app.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/test_domain_cross_domain/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/test_domain_cross_domain.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/test_domain_unified_attribution/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/test_domain_unified_attribution.csv".format(date),header = 'true')
# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/mw_basic_kpi/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/mw_basic_kpi.csv".format(date),header = 'true')




In [0]:
%%sh

# rm -rf  /tmp/basic_kpi.csv/  /tmp/audience.csv/  /tmp/demographics.csv/  /tmp/crossapps.csv/ /tmp/appused.csv/

DATE='2021-08-13'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/basic_kpi.csv/ /tmp/$DATE/basic_kpi.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/audience.csv/ /tmp/$DATE/audience.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/demographics.csv/ /tmp/$DATE/demographics.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/crossapps.csv/ /tmp/$DATE/crossapps.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/appused.csv/ /tmp/$DATE/appused.csv/

# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/domain_outbound.csv/ /tmp/$DATE/domain_outbound.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/domain_referral.csv/ /tmp/$DATE/domain_referral.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/test_app_cross_domain.csv/ /tmp/$DATE/test_app_cross_domain.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/test_domain_cross_app.csv/ /tmp/$DATE/test_domain_cross_app.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/test_domain_cross_domain.csv/ /tmp/$DATE/test_domain_cross_domain.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/test_domain_unified_attribution.csv/ /tmp/$DATE/test_domain_unified_attribution.csv/
# aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.tom.s6usage/adhoc/$DATE/mw_basic_kpi.csv/ /tmp/$DATE/mw_basic_kpi.csv/



echo "=======basic_kpi========="
cat /tmp/$DATE/basic_kpi.csv/*.csv
# echo "========audience========"
# cat /tmp/$DATE/audience.csv/*.csv
# echo "======demographics=========="
# cat /tmp/$DATE/demographics.csv/*.csv
# echo "=====crossapps==========="
# cat /tmp/$DATE/crossapps.csv/*.csv
# echo "======appused=========="
# cat /tmp/$DATE/appused.csv/*.csv


# echo "======domain_outbound=========="
# cat /tmp/$DATE/domain_outbound.csv/*.csv
# echo "======domain_referral=========="
# cat /tmp/$DATE/domain_referral.csv/*.csv
# echo "======test_app_cross_domain=========="
# cat /tmp/$DATE/test_app_cross_domain.csv/*.csv
# echo "======test_domain_cross_app=========="
# cat /tmp/$DATE/test_domain_cross_app.csv/*.csv
# echo "======test_domain_cross_domain=========="
# cat /tmp/$DATE/test_domain_cross_domain.csv/*.csv
# echo "======test_domain_unified_attribution=========="
# cat /tmp/$DATE/test_domain_unified_attribution.csv/*.csv
# echo "======mw_basic_kpi=========="
# cat /tmp/$DATE/mw_basic_kpi.csv/*.csv

In [0]:


import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-06-03", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/audience/version=fix_crow_1.0.0/range_type={range_type}/date={date}/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_audience")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type,count(distinct(age)) as count_age, count(distinct(gender)) as count_gender, sum(IDX) as sum from test_audience".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-08-13/audience/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))

In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-06-03", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/crossapps/version=fix_crow_1.0.0/range_type={range_type}/date={date}/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_crossapps")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type,count(distinct(selected_app)) as count_selected_app, sum(CROSSAPP_UP) as sum from test_crossapps".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-08-13/crossapps/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))

In [0]:


import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-06-03", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/appused/version=fix_crow_1.0.0/range_type={range_type}/date={date}/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_appused")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type,count(distinct(selected_app)) as count_selected_app, sum(AU) as sum from test_appused".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-08-13/appused/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))


In [0]:


import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-06-03", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
        try:
            s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=fix_crow_1.0.0/range_type={range_type}/date={date}/".format(range_type=range_type_mapping[granularity], date=date)
            spark.read.parquet(s3path).createOrReplaceTempView("test_demographics")
            df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type,count(distinct(age)) as count_age, count(distinct(gender)) as count_gender, sum(AU) as sum from test_demographics".format(date=date, granularity=granularity))
            df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-08-13/demographics/", mode="append") #append
            print("PASS on {} {}".format(granularity, date))
        except Exception as e:
            print("ERROR on {} {}".format(granularity, date))


In [0]:

# e2e basic
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=fix_crow_1.0.0/range_type=WEEK/date=2021-05-08/").createOrReplaceTempView("test")
spark.sql("select * from test where country='ID' and app_id = 284882215").show(20)

In [0]:

# e2e app used
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/to_tech/appused/version=fix_crow_1.0.0/range_type=MONTH/date=2021-05-31/").createOrReplaceTempView("test")
spark.sql("select * from test where country='HK' and app_id = 284882215").show(20)

In [0]:

# e2e cross app
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/to_tech/crossapps/version=fix_crow_1.0.0/range_type=MONTH/date=2021-05-31/").createOrReplaceTempView("test")
spark.sql("select * from test where country='ID' and app_id = 284882215").show(20)

In [0]:

# e2e audience
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/to_tech/audience/version=fix_crow_1.0.0/range_type=MONTH/date=2021-05-31/").createOrReplaceTempView("test")
spark.sql("select * from test where country='ID' and app_id = 284882215").show(20)

In [0]:

# e2e demographics
spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=fix_crow_1.0.0/range_type=MONTH/date=2021-05-31/").createOrReplaceTempView("test")
spark.sql("select * from test where country='HK' and app_id = 284882215").show(20)

In [0]:

date='2021-10-02'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/retention_month/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/retention_month.csv".format(date),header = 'true')

In [0]:
%%sh


DATE='2021-10-02'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/retention_month.csv/ /tmp/$DATE/retention_month.csv/



echo "=======retention_week========="
cat /tmp/$DATE/retention_month.csv/*.csv


In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}
metric_name_week = ["RRW0", "RRW1", "RRW2", "RRW3", "RRW4","RRW5","RRW6","RRW7","RRW8","RRW9","RRW10","RRW11","RRW12","RRW13","RRW14","RRW15","RRW16","RRW17","RRW18","RRW19","RRW20"]

for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}/".format(date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_retention_week")
                df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(value) as sum from test_retention_week where value is not null".format(date=date, granularity=granularity))
                # df.show(10)
                # df = spark.sql("select distinct(device_type) from test_retention_week")
                # df.show(100)
                # df = spark.sql("select * from test_retention_week limit 20")
                # print(df)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-10-02/retention_month/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))

In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}/".format(date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_retention_day")
                df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(value) as sum from test_retention_week where value is not null".format(date=date, granularity=granularity))
                df.show(100)
                # df = spark.sql("select distinct(device_type) from test_retention_day")
                # df.show(100)
                # df = spark.sql("select * from test_retention_week limit 20")
                # print(df)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-09-29/retention_day/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))

In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "weekly": get_date_list(begin_date, end_date, "W-SAT")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2021-08-27", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}
metric_name_week = ["RRW0", "RRW1", "RRW2", "RRW3", "RRW4","RRW5","RRW6","RRW7","RRW8","RRW9","RRW10","RRW11","RRW12","RRW13","RRW14","RRW15","RRW16","RRW17","RRW18","RRW19","RRW20"]

for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}/".format(date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_retention_week")
                df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, sum(value) as sum from test_retention_week where value is not null".format(date=date, granularity=granularity))
                df.show(10)
                # df = spark.sql("select distinct(device_type) from test_retention_week")
                # df.show(100)
                # df = spark.sql("select * from test_retention_week limit 20")
                # print(df)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2021-10-01/retention_week/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))

In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date=2021-03-31/metric_name=RRW

In [0]:

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date=2020-08-31/").createOrReplaceTempView("test")
spark.sql("select * from test").show(20)

In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/usage_week_cohort_retention/version=v1.0.0/range_type=WEEK/date=2021-06-05/metric_name=RRD0/platform=1/


In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-12-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-02-01", '%Y-%m-%d')

range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
#s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}
#s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D7_routine/version=v1.0.0/range_type=MONTH/date={date}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/demographics/version=1.0.0/range_type={range_type}/date={date}/".format(range_type=range_type_mapping[granularity], date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_demographic")
                df = spark.sql("select '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device, count(distinct(age)) as age, count(distinct(gender)) as gender, sum(AU) as sum from test_demographic".format(date=date, granularity=granularity))
                # df.show(100)
                #df = spark.sql("select * from test_demographic_sample")
                # df.show(100)
                # df = spark.sql("select distinct(count_device) from test_demographic_week where age=30")
                #df = spark.sql("select app_id, device_type, country, age, gender, AU, platform from test_demographic_week WHERE app_id is null or device_type is null or country is null or age is null or gender is null or AU is null or platform is null")
                #df.show(20)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-02-01/demographics/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))


In [0]:

date='2022-02-01'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/demographics/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/demographics.csv".format(date),header = 'true')


In [0]:
%%sh


DATE='2022-02-01'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/demographics.csv/ /tmp/$DATE/demographics.csv/



echo "=======Demographics========="
cat /tmp/$DATE/demographics.csv/*.csv


In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-12-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-02-01", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
#s3://aardvark-prod-dca-data/oss/USAGE_BASIC_KPI/
#s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}
#s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D7_routine/version=v1.0.0/range_type=MONTH/date={date}
#s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D14_routine/version=v1.0.0/range_type=MONTH/date={date}
#s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D30_routine/version=v1.0.0/range_type=MONTH/date={date}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-dca-data/oss/GAMEIQ_GENRE_USAGE_AGE_GENDER_RELEASE_KPI/version=v2.0.0/range_type=MONTH/date={date}/".format(date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("full_gameiq")
                df = spark.sql("select '{granularity}' as granularity, '{date}' as date, count(distinct(genre_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device, count(distinct(age)) as count_age, count(distinct(gender)) as count_gender, sum(value) as pct_sum from full_gameiq where metric_name='PCT' and genre_id between 2000000 and 2999999 or genre_id  between 2000 and 2999".format(date=date, granularity=granularity))
                # df.show(100)
                # df = spark.sql("select count(distinct(country)) as count_country from test_genre_level")
                # df.show(100)
                # df = spark.sql("select genre_id, device_type, country, age, gender, value, metric_name, platform from test_genre_level WHERE genre_id is null or device_type is null or country is null or age is null or gender is null or value is null or metric_name is null or value is null")

                df.show(100)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-02-04/full_game_iq/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))



In [0]:

date='2022-02-04'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/full_game_iq/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/full_game_iq.csv".format(date),header = 'true')


In [0]:
%%sh


DATE='2022-02-04'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/full_game_iq.csv/ /tmp/$DATE/full_game_iq.csv/



echo "=======retention_week========="
cat /tmp/$DATE/full_game_iq.csv/*.csv


In [0]:

df = spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/usage_week_cohort_retention/version=v1.0.0/range_type=WEEK/date=2021-12-18/metric_name=RRM0/").filter("app_id=20600000009072 and country='US'")
df.show(20)

In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        # "monthly": get_date_list(begin_date, end_date, "M"),
        "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-06-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-01-31", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
#s3://aardvark-prod-pdx-mdm-to-int/usage_long_term_retention/version=v1.0.0/range_type=MONTH/date={date}
#s3://aardvark-prod-pdx-mdm-to-int/usage_retention/batch=retention_batch_D7_routine/version=v1.0.0/range_type=MONTH/date={date}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-pdx-mdm-to-int/usage_week_cohort_retention/version=v1.0.0/range_type=WEEK/date={date}/".format(date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_weekly_cohorts_full")
                df = spark.sql("select  '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device_type, count(distinct(platform)) as platform, sum(value) as sum from test_weekly_cohorts_full where value is not null".format(date=date, granularity=granularity))
                df.show(100)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-01-20/weekly_cohorts_full/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))


In [0]:

date='2022-01-20'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/weekly_cohorts_full/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/weekly_cohorts_full.csv".format(date),header = 'true')


In [0]:
%%sh


DATE='2022-01-20'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/weekly_cohorts_full.csv/ /tmp/$DATE/weekly_cohorts_full.csv/



echo "=======retention_week========="
cat /tmp/$DATE/weekly_cohorts_full.csv/*.csv

In [0]:
%%sh
aws s3 ls s3://aardvark-prod-dca-data/oss/GAMEIQ_GENRE_USAGE_AGE_GENDER_RELEASE_KPI/version=v2.0.0/range_type=MONTH/date=2022-01-31/platform=1/metric_name=IDX/



In [0]:

spark.read.parquet("s3://aardvark-prod-dca-data/oss/GAMEIQ_GENRE_USAGE_AGE_GENDER_RELEASE_KPI/version=v2.0.0/range_type=MONTH/date=2022-01-31/").createOrReplaceTempView("test")
#spark.sql("select * from test WHERE app_id is null or device_type is null or country is null or age is null or gender is null or value is null or metric_name is null or platform is null").show(100)
spark.sql("select * from test").show(20)
#spark.sql("select * from test WHERE app_id is null or device_type is null or country is null or age is null or gender is null or IDX is null or PCT is null or platform is null").show(100)


In [0]:

import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "monthly": get_date_list(begin_date, end_date, "M"),
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-12-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-02-01", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}


for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:
        
            try:
                s3path = "s3://aardvark-prod-pdx-mdm-to-int/to_tech/audience/version=1.0.0/range_type=MONTH/date={date}/".format(date=date)
                spark.read.parquet(s3path).createOrReplaceTempView("test_audience")
                df = spark.sql("select '{granularity}' as granularity, '{date}' as date, count(distinct(app_id)) as count_app_id, count(distinct(country)) as count_country, count(distinct(device_type)) as count_device, count(distinct(age)) as count_age, count(distinct(gender)) as count_gender, sum(IDX) as idx_sum, sum(PCT) as pct_sum from test_audience".format(date=date, granularity=granularity))
                # df.show(100)
                # df = spark.sql("select count(distinct(country)) as count_country from test_genre_level")
                # df.show(100)
                # df = spark.sql("select app_id, device_type, country, age, gender, value, metric_name, platform from test_genre_level WHERE app_id is null or device_type is null or country is null or age is null or gender is null or value is null or metric_name is null or platform is null")

                # df.show(100)
                df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-02-01/age_bins_audience/", mode="append") #append
                print("PASS on {} {}".format(granularity, date))
            except Exception as e:
                print("ERROR on {} {}".format(granularity, date))

In [0]:

date='2022-02-01'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/age_bins_audience/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/age_bins_audience.csv".format(date),header = 'true')

In [0]:
%%sh


DATE='2022-02-01'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/age_bins_audience.csv/ /tmp/$DATE/age_bins_audience.csv/



echo "=======retention_week========="
cat /tmp/$DATE/age_bins_audience.csv/*.csv


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-ugc/unified


In [0]:
%%sh
